In [ ]:
import sionna
import tensorflow as tf

# Import Sionna RT components
from sionna.rt import load_scene, PlanarArray, Receiver, Transmitter

In [ ]:
seed = 1234

# Load integrated scene

scene_name = "munich"

if scene_name == "simple":
    scene = load_scene(sionna.rt.scene.simple_street_canyon)
elif scene_name == "etoile":
    scene = load_scene(sionna.rt.scene.etoile)
elif scene_name == "munich":
    scene = load_scene(sionna.rt.scene.munich)
else:
    raise ValueError(f"Unknown scene '{scene_name}'")

# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=1, 
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="cross")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="cross")

# Too large values can cause OOM errors (or lags) on
# larger scenes.
n_sources = 10
n_targets = 10

min_x, min_y, min_z = scene.mi_scene.bbox().min
max_x, max_y, max_z = scene.mi_scene.bbox().max

max_z += 10

for i, x, y, z in zip(
    range(n_sources),
    tf.random.uniform((n_sources,), minval=min_x, maxval=max_x, seed=seed),
    tf.random.uniform((n_sources,), minval=min_y, maxval=max_y, seed=seed),
    tf.random.uniform((n_sources,), minval=min_z, maxval=max_z, seed=seed),
):
    # Create transmitter
    tx = Transmitter(name=f"tx_{i}",
                     position=[x, y, z])

    # Add transmitter instance to scene
    scene.add(tx)
    
for i, x, y, z in zip(
    range(n_targets),
    tf.random.uniform((n_targets,), minval=min_x, maxval=max_x, seed=seed),
    tf.random.uniform((n_targets,), minval=min_y, maxval=max_y, seed=seed),
    tf.random.uniform((n_targets,), minval=min_z, maxval=max_z, seed=seed),
):
    # Create receiver
    rx = Receiver(name=f"rx_{i}",
                  position=[x, y, z])

    # Add receiver instance to scene
    scene.add(rx)

scene.preview()

In [ ]:
# Compute propagation paths
paths = scene.compute_paths(max_depth=2,
                            #method="exhaustive",
                            method="stochastic", # For small scenes the method can be also set to "exhaustive"
                            num_samples=1e6,     # Number of rays shot into random directions, too few rays can lead to missing paths
                            seed=seed)           # By fixing the seed, reproducible results can be ensured

scene.preview(paths, show_devices=True, show_paths=True) # Use the mouse to focus on the visualized paths

In [ ]:
# [max_depth, num_targets, num_sources, max_num_paths, coordinates : 3]
# max_num_paths cannot be predicted, but is always <= num_primitives**max_depth + 1
paths.vertices